# 1. read predictions
# 2. read references
# 3. analyze na_score and start_end_score for NA and SA seperately (and LA?)

In [1]:
import pandas as pd
import json

evaluation_directory = "/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/exclude_passage_answers_discarddups_la_as_sa_moreneg0208_2wayloss/eval_normalized/output/"
evaluation_directory = "/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/exclude_passage_answers_discarddups_la_as_sa_2wayloss/eval_normalized/output/"
evaluation_directory = "/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/exclude_passage_answers_discarddups_la_as_sa_moreneg0416_2wayloss/output/"

def load_dir(evaluation_directory):
    with open(evaluation_directory + "eval_predictions.json") as fi:
        predictions = json.load(fi)
    references = pd.read_json(evaluation_directory + "eval_references.json")
    return predictions, references

In [48]:
references.sample(2)

,start_position,end_position,passage_index,yes_no_answer,example_id,language
4149,"[47709, 47709, 47728, 47709, 47709]","[47714, 47714, 47733, 47719, 47719]","[17, 17, 19, 17, 17]","[0, 0, 0, 0, 0]","[-4054205284200755552, -4054205284200755552, -...","[english, english, english, english, english]"
429,"[48696, 52536, 48861, 42554, 48696]","[48711, 52551, 48878, 42583, 48711]","[40, 42, 40, 0, 40]","[0, 0, 0, 0, 0]","[1838851770314085590, 1838851770314085590, 183...","[english, english, english, english, english]"


In [2]:
def is_sa(list):
    na_count = 0
    for item in list:
        if item == -1:
            na_count += 1
    if na_count >= 3:
        return False
    return True

In [11]:
import numpy 

def compute_analysis(predictions, references, offset=1):
    reference_count = {'na':0, 'sa':0, 'la':0, 'yn':0}
    scores = {}
    scores['sa'] = {}
    # scores['la'] = {}
    scores['na'] = {}
    # scores['yn'] = {}
    predict_na = {'tp':0,'fp':0,'tn':0,'fn':0}

    score_types = ['cls_score', 'start_logit', 'end_logit', 'start+end']

    for score in scores:
        for score_type in score_types:
            scores[score][score_type] = []

    for i, reference in references.iterrows():
        # print(reference)
        example_id = reference['example_id'][0]
        # squad:
        # example_id = reference['id']
        answer_type = 'na'
        
        # squad
        # if reference['answers']['answer_start'][0] == -1:
        #     answer_type = 'na'
        # nq
        # if is_sa(reference['start_position']):
        if reference['start_position'][0] > -1:
            answer_type = 'sa'
        # elif is_sa(reference['passage_index']):
        #     answer_type = 'la'
        # elif sum(reference['yes_no_answer']) > 0:
        #     answer_type = 'yn'
        reference_count[answer_type] += 1
        # print(answer_type)
        prediction = predictions[example_id]
        scores[answer_type]['cls_score'].append(prediction[0]['cls_score'][offset])
        scores[answer_type]['start_logit'].append(prediction[0]['start_logit'][offset])
        scores[answer_type]['end_logit'].append(prediction[0]['end_logit'][offset])
        scores[answer_type]['start+end'].append((prediction[0]['start_logit'][offset] + prediction[0]['end_logit'][offset]))

        predict_as_na = False
        if prediction[0]['start_logit'][offset] + prediction[0]['end_logit'][offset] < prediction[0]['cls_score'][offset]:
            predict_as_na = True

        if predict_as_na and answer_type == 'na':
            predict_na['tn'] += 1
        elif predict_as_na:
            predict_na['fn'] += 1
        elif not predict_as_na and answer_type == 'sa' or answer_type == 'la':
            predict_na['tp'] += 1
        else:
            predict_na['fp'] += 1

    for score in scores:
        for score_type in scores[score]:
            mean = numpy.average(scores[score][score_type])
            min = numpy.min(scores[score][score_type])
            max = numpy.max(scores[score][score_type])
            print(f'{score} {score_type} mean: {mean}, min: {min}, max: {max}')

    print(predict_na)
    print(reference_count)

    p = predict_na['tp']/(predict_na['tp']+predict_na['fp'])
    r = predict_na['tp']/(predict_na['tp']+predict_na['fn'])
    f1 = 2 * p*r / (p + r )
    print(f'f1: {f1}, p: {p}, r {r}')

In [14]:
# default neg
compute_analysis(predictions, references, 0)

sa cls_score mean: 9.295011520933832, min: 5.035934925079346, max: 15.761591911315918
sa start_logit mean: 9.143398548268724, min: 1.620445966720581, max: 11.391331672668457
sa end_logit mean: 9.29574817308007, min: -0.06604782491922379, max: 11.564820289611816
sa start+end mean: 18.439146721348795, min: 2.926615558564663, max: 22.870535850524902
la cls_score mean: 9.793096344629161, min: 3.9367176294326782, max: 15.714771747589111
la start_logit mean: 7.860998722312929, min: -1.17532217502594, max: 11.34286117553711
la end_logit mean: 7.321585907638788, min: -0.40927380323410034, max: 11.391169548034668
la start+end mean: 15.182584629951716, min: 4.279646575450897, max: 22.54685401916504
na cls_score mean: 11.661749788675552, min: 5.95200252532959, max: 18.17583179473877
na start_logit mean: 6.209973320643871, min: -4.618706226348877, max: 11.37607479095459
na end_logit mean: 5.808325404800379, min: -2.2680604457855225, max: 11.423245429992676
na start+end mean: 12.01829872544425, min

In [12]:
# more neg 02 08
compute_analysis(predictions, references, 0)

sa cls_score mean: 8.996568983039637, min: 2.9522416591644287, max: 16.08403778076172
sa start_logit mean: 8.383533713232968, min: 1.7111743688583374, max: 10.603812217712402
sa end_logit mean: 8.640907535881832, min: 2.442873239517212, max: 10.991469383239746
sa start+end mean: 17.0244412491148, min: 4.904268145561218, max: 21.481185913085938
la cls_score mean: 9.902557944794067, min: 4.289251327514648, max: 15.248830318450928
la start_logit mean: 7.191161243178004, min: -0.9893350601196289, max: 10.985502243041992
la end_logit mean: 6.741889385040477, min: -1.1990559101104736, max: 10.598464012145996
la start+end mean: 13.933050628218481, min: 1.4623978435993195, max: 21.041714668273926
na cls_score mean: 11.850911533893683, min: 4.636003494262695, max: 17.275089263916016
na start_logit mean: 5.446020014877312, min: -3.7662668228149414, max: 10.621685981750488
na end_logit mean: 5.128890119943863, min: -3.339244842529297, max: 10.642224311828613
na start+end mean: 10.574910134821176,

In [10]:
# more neg 02 08
compute_analysis(predictions, references, 1)

sa cls_score mean: 0.034507753130909806, min: 0.000269461830612272, max: 0.9868164956569672
sa start_logit mean: 0.6656259664735343, min: 0.0007884519873186946, max: 0.9982413649559021
sa end_logit mean: 0.6842875601238949, min: 0.0017802660586312413, max: 0.9945623874664307
sa start+end mean: 0.6749567632987146, min: 0.008628678508102894, max: 0.9950990974903107
la cls_score mean: 0.07437061898513707, min: 0.0006594950100407004, max: 0.9174738526344299
la start_logit mean: 0.48053132619334776, min: 0.00014855837798677385, max: 0.9965887665748596
la end_logit mean: 0.35102770037983205, min: 3.4014716220553964e-05, max: 0.990546703338623
la start+end mean: 0.4157795132865899, min: 0.00876842881552875, max: 0.9875251650810242
na cls_score mean: 0.41497283293932785, min: 0.0014385844115167856, max: 0.9995205998420715
na start_logit mean: 0.31220777307330827, min: 6.636083980993135e-06, max: 0.9946584105491638
na end_logit mean: 0.2679373626426588, min: 1.1241228094149847e-05, max: 0.99227

In [16]:
# more neg (08 16)
compute_analysis(predictions, references, 0)

sa cls_score mean: 12.55491405556485, min: 7.295560598373413, max: 18.194772720336914
sa start_logit mean: 9.291254094411473, min: -1.6602362394332886, max: 11.847705841064453
sa end_logit mean: 9.60330882481517, min: -1.5028188228607178, max: 12.084779739379883
sa start+end mean: 18.894562919226644, min: -3.1630550622940063, max: 23.8850040435791
la cls_score mean: 13.331995268805708, min: 8.433923721313477, max: 17.892271041870117
la start_logit mean: 7.754471804800747, min: -2.5014665126800537, max: 11.820550918579102
la end_logit mean: 7.330774385550564, min: -2.283249616622925, max: 12.67302131652832
la start+end mean: 15.08524619035131, min: -0.4836796522140503, max: 23.770538330078125
na cls_score mean: 15.034318093030882, min: 9.29923963546753, max: 19.30993938446045
na start_logit mean: 5.770045666606762, min: -7.987424373626709, max: 12.079704284667969
na end_logit mean: 5.468224812119435, min: -6.801821231842041, max: 11.736608505249023
na start+end mean: 11.238270478726198,

In [12]:
import glob

files = glob.glob("/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/exclude_passage_answers_discarddups_la_as_sa_*")

for file in files:
    if "04100" in file:
        continue
    if '12' in file:
        file = file + '/output/'
    else:
        file = file + "/eval/output/"
    print(file)
    predictions, references = load_dir(file)
    compute_analysis(predictions,references)

/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/exclude_passage_answers_discarddups_la_as_sa_moreneg1040_2waylossv2/eval/output/
{'tp': 3552, 'fp': 1235, 'tn': 2689, 'fn': 354}
f1: 0.8172092488208904, p: 0.7420096093586798, r 0.9093701996927803
/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/exclude_passage_answers_discarddups_la_as_sa_moreneg0624_2wayloss/eval/output/
{'tp': 3649, 'fp': 1529, 'tn': 2402, 'fn': 250}
f1: 0.8040101355073263, p: 0.7047122441096949, r 0.9358809951269557
/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/exclude_passage_answers_discarddups_la_as_sa_moreneg1250_2wayloss/output/
{'tp': 3458, 'fp': 1056, 'tn': 2866, 'fn': 450}
f1: 0.8211826169555926, p: 0.7660611431103235, r 0.8848515864892528
/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/exclude_passage_answers_discarddups_la_as_sa_moreneg0832_2wayloss/eval/output/
{'tp': 3597, 'fp': 1421, 'tn': 2506, 'fn': 306}
f1: 0.8064118372379

In [5]:
import glob

files = glob.glob("/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/NQftSAP_1040_lr4e5new*")

for file in files:
    file = file + "/output/"
    print(file)
    predictions, references = load_dir(file)
    compute_analysis(predictions,references)

/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/NQftSAP_1040_lr4e5new/output/
sa cls_score mean: 0.03040216642581254, min: 5.833195018567494e-06, max: 0.9999338090419769
sa start_logit mean: 0.5022178653243818, min: 9.498157851339784e-06, max: 0.998292088508606
sa end_logit mean: 0.5175571929046983, min: 4.806697688763961e-06, max: 0.9998937249183655
sa start+end mean: 1.01977505822908, min: 1.955812786036404e-05, max: 1.9971556663513184
na cls_score mean: 0.9921587024815198, min: 2.9660641303053126e-05, max: 0.9999961853027344
na start_logit mean: 0.0034236825543650902, min: 2.7985166184407717e-08, max: 0.9927844405174255
na end_logit mean: 0.003602054981494753, min: 2.2240143948693003e-08, max: 0.9943963885307312
na start+end mean: 0.007025737535859843, min: 1.392824401591497e-06, max: 1.9871808290481567
{'tp': 979, 'fp': 6, 'tn': 994, 'fn': 21}
{'na': 1000, 'sa': 1000, 'la': 0, 'yn': 0}
f1: 0.9863979848866499, p: 0.9939086294416244, r 0.979
/dccstor/srosent2/prim

In [12]:
predictions, references = load_dir("/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/NQftSAP_1040_lr4e5eval2/output/")
compute_analysis(predictions,references)

sa cls_score mean: 0.00027519666183812983, min: 7.370597465694573e-08, max: 0.06079230950126657
sa start_logit mean: 0.528337646078937, min: 7.390306109300582e-07, max: 0.9997084140777588
sa end_logit mean: 0.46912605489077897, min: 6.0692927945638075e-06, max: 0.9999433755874634
sa start+end mean: 0.9974637009697159, min: 0.03397626802325249, max: 1.9941377639770508
na cls_score mean: 5.055322854992106e-06, min: 5.1320013128020037e-08, max: 0.0022160150642207554
na start_logit mean: 0.54821919599858, min: 8.340289809893875e-07, max: 0.9999852776527405
na end_logit mean: 0.4475774540861271, min: 2.0547254280245397e-06, max: 0.9999808669090271
na start+end mean: 0.9957966500847072, min: 3.818596997007262e-05, max: 1.9997809529304504
{'tp': 1000, 'fp': 1000, 'tn': 0, 'fn': 0}
{'na': 1000, 'sa': 1000, 'la': 0, 'yn': 0}
f1: 0.6666666666666666, p: 0.5, r 1.0


In [13]:
import glob

files = glob.glob("/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/SAP/nq_format/*")

for file in files:
    file = file + "/output/"
    print(file)
    predictions, references = load_dir(file)
    compute_analysis(predictions,references)

/dccstor/srosent2/primeqa/experiments/reader_reranker/2epochRoberta/SAP/nq_format/NQftSAP_lr4e-5_2ep-0.50negs/output/
sa cls_score mean: 0.3124020789358619, min: 0.00014523408754030243, max: 0.9999945163726807
sa start_logit mean: 0.34276265454581595, min: 5.131719404971591e-08, max: 0.996731698513031
sa end_logit mean: 0.28781373914371566, min: 7.460797490921323e-08, max: 0.9969581365585327
sa start+end mean: 0.6305763936895317, min: 1.1063960130286432e-06, max: 1.9935619831085205
na cls_score mean: 0.930867491889745, min: 0.0737343356013298, max: 0.9999984204769135
na start_logit mean: 0.03237353638353954, min: 1.2912174440771196e-08, max: 0.9289714694023132
na end_logit mean: 0.02808201828511829, min: 1.0379930692749895e-08, max: 0.8902056217193604
na start+end mean: 0.06045555466865784, min: 3.667508607918535e-08, max: 1.8191770911216736
{'tp': 700, 'fp': 65, 'tn': 935, 'fn': 300}
{'na': 1000, 'sa': 1000, 'la': 0, 'yn': 0}
f1: 0.7932011331444758, p: 0.9150326797385621, r 0.7
/dccst